In [ ]:
#############################################

In [ ]:
import paramiko
import time
import sys

ip = "172.16.16.122"
user = "root"
pw = "asd123!@"

# ---- 원격 실행 및 실시간 출력 함수 ----
def run_remote(script):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, password=pw, port=22)

    shell = ssh.invoke_shell()
    shell.send(script + "\n")

    while True:
        if shell.recv_ready():
            output = shell.recv(4096).decode('utf-8', errors='ignore')
            print(output, end='')
            sys.stdout.flush()
        # exit_status_ready는 단순 명령에는 바로 True가 되지 않으므로, timeout 기반 루프
        if shell.closed:
            break
        time.sleep(0.1)

    ssh.close()

def dhcp() : ## 완료 
    return """

    Rocky9 DHCP 자동 스크립트
    
    
    
    #!/bin/bash
    # Rocky 9 DHCP 자동 설치 & 기본 설정 자동화
    
    # 1.root 권한 확인
    if [[ $EUID -ne 0 ]]; then
        echo "이 스크립트는 root 권한으로 실행해야 합니다."
        exit 1
    fi
    
    # 2️.DHCP 서버 설치
    dnf install -y dhcp-server
    
    # 3️.DHCP 설정 경로 확인
    DHCP_CONF="/etc/dhcp/dhcpd.conf"
    
    # 4️.기존 설정 백업
    if [[ -f $DHCP_CONF ]]; then
        cp $DHCP_CONF "${DHCP_CONF}.bak_$(date +%F_%H%M%S)"
        echo "기존 dhcpd.conf 백업 완료"
    fi
    
    # 5️.기본 DHCP 설정 작성
    cat > $DHCP_CONF <<EOF
    # 기본 DHCP 설정 - 자동화 스크립트
    default-lease-time 600;
    max-lease-time 7200;
    authoritative;
    
    # 예제 서브넷
    subnet 172.16.0.0 netmask 255.255.0.0 {
        range 172.16.254.6 172.16.254.200;
        option routers 172.16.0.1;
        option subnet-mask 255.255.0.0;
        option domain-name-servers 8.8.8.8, 8.8.4.4;
    }
    EOF
    
    echo " DHCP 설정 파일 작성 완료: $DHCP_CONF"
    echo " DHCP 포트 허용 완료"
    
    
    # 6.DHCP 서비스 활성화 & 시작
    systemctl enable --now dhcpd
    
    # 7.상태 확인
    echo "==== DHCP 서비스 상태 ===="
    systemctl status dhcpd --no-pager
    
    echo " DHCP 서버 설치 및 기본 설정 완료!"

"""

def lamp_ftp() : ## 완료
    return """
    #!/bin/bash
set -e

echo "===== Rocky Linux 서버 초기화 스크립트 시작 ====="

# 0️⃣ firewalld 비활성화
echo "firewalld 비활성화 중..."
systemctl stop firewalld
systemctl disable firewalld
echo "firewalld 비활성화 완료!"


# 3️⃣ Apache 설치
echo "Apache 설치 중..."
dnf -y install httpd
systemctl enable --now httpd
echo "Apache 설치 및 자동 시작 완료!"

# 4️⃣ 사용자 웹 디렉토리 생성
WEB_DIR="/home/heeyoung"
mkdir -p "$WEB_DIR"
chmod 755 "$WEB_DIR"
chown -R $USER:$USER "$WEB_DIR"

# SELinux 활성화 시 컨텍스트 적용
if sestatus | grep -q "enabled"; then
    chcon -R -t httpd_sys_content_t "$WEB_DIR"
fi
echo "웹 디렉토리 생성 및 권한 설정 완료!"

# 5️⃣ VirtualHost 설정
VHOST_CONF="/etc/httpd/conf.d/heeyoung.conf"
cat <<EOF > "$VHOST_CONF"
<VirtualHost *:80>
    ServerName www.heeyoung.com
    DocumentRoot "$WEB_DIR"
    <Directory "$WEB_DIR">
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>
</VirtualHost>
EOF
echo "VirtualHost 설정 완료: $VHOST_CONF"

# 6️⃣ index.html 생성
echo "qqq" > "$WEB_DIR/index.html"
echo "index.html 생성 완료!"

# 7️⃣ Apache 재시작
systemctl restart httpd
echo "Apache 재시작 완료!"

# 8️⃣ vsftpd 설치 및 시작
echo "vsftpd 설치 중..."
dnf -y install vsftpd
systemctl enable --now vsftpd
echo "vsftpd 설치 및 자동 시작 완료!"

echo "🎉 모든 설정 완료! 웹 서버와 FTP 서버가 준비되었습니다."

    """

def dns_setiing1() :
     return """
        #!/bin/bash
  set -e

echo "=== 1. BIND (DNS) 설치 ==="
dnf -y install bind bind-utils

echo "=== 2. named 설정 파일 수정 ==="
sed -i 's/listen-on[[:space:]]*port 53[[:space:]]*{[[:space:]]*127.0.0.1;[[:space:]]*};/listen-on port 53 { any; };/g' /etc/named.conf
sed -i 's/allow-query[[:space:]]*{[[:space:]]*localhost;[[:space:]]*};/allow-query { any; };/g' /etc/named.conf

echo "=== 3. 존 파일 등록 ==="
# 중복 방지: 기존 내용 제거 후 새로 작성
cat <<EOF >> /etc/named.rfc1912.zones
zone "team1.com" IN {
    type master;
    file "team1.com.zone";
    allow-update { none; };
};
EOF


    echo "=== 4. Apache 가상호스트 설정 ==="
    cat <<EOF > /etc/httpd/conf.d/team1.conf
    <VirtualHost *:80>
        ServerName ww1.team1.com
        DocumentRoot "/home/team1"
        <Directory "/home/team1/">
            AllowOverride None
            Options Indexes FollowSymLinks
            Require all granted
        </Directory>
    </VirtualHost>

    <VirtualHost *:80>
        ServerName ww2.team1.com
        DocumentRoot "/home/team2"
        <Directory "/home/team2/">
            AllowOverride None
            Options Indexes FollowSymLinks
            Require all granted
        </Directory>
    </VirtualHost>
    EOF

    echo "=== 5. 웹 디렉토리 생성 ==="
    mkdir -p /home/team1 /home/team2
    echo "<h1>Welcome to ww1.team1.com</h1>" > /home/team1/index.html
    echo "<h1>Welcome to ww2.team1.com</h1>" > /home/team2/index.html
    chown -R apache:apache /home/team1 /home/team2
    echo "=== 4. 존 파일 생성 ==="
    #cp -ap /var/named/named.localhost /var/named/team1.com.zone
    
"""    

    
def php_mariaDB() :
    #!/bin/bash
    return """set -e
    
    echo "===== LAMP (Apache + PHP + MariaDB + phpMyAdmin) 자동 설치 시작 ====="
    
    # 0️⃣ SELinux & firewalld 비활성화
    echo ">>> SELinux & firewalld 비활성화 중..."
    sed -i 's/^SELINUX=.*/SELINUX=disabled/' /etc/selinux/config
    systemctl stop firewalld
    systemctl disable firewalld
    echo "✅ SELinux 비활성화 (재부팅 후 적용), firewalld 중지 완료!"
    
    # 1️⃣ 필수 패키지 설치
    echo ">>> DNF 플러그인 설치..."
    dnf -y install dnf-plugins-core epel-release expect
    
    # 2️⃣ PHP 리포지토리 등록 및 활성화
    echo ">>> Remi 리포지토리 설정 중..."
    dnf install -y https://rpms.remirepo.net/enterprise/remi-release-9.rpm
    dnf module reset php -y
    dnf module enable php:remi-8.4 -y
    echo "✅ PHP 8.4 모듈 활성화 완료!"
    
    # 3️⃣ Apache + PHP 설치
    echo ">>> Apache + PHP 설치 중..."
    dnf -y install httpd php php-fpm php-mysqlnd php-cli php-common php-devel \
        php-mbstring php-xml php-gd php-json php-intl php-zip php-opcache
    
    systemctl enable --now httpd php-fpm
    echo "✅ Apache & PHP 설치 및 자동 시작 완료!"
    
    # 4️⃣ MariaDB 설치 및 자동 실행
    echo ">>> MariaDB 설치 중..."
    dnf install -y mariadb-server
    systemctl enable --now mariadb
    echo "✅ MariaDB 설치 및 실행 완료!"
    
    # 5️⃣ MariaDB 초기화 (expect 자동화)
    MYSQL_ROOT_PASSWORD="asd123!@"
    
    cat <<'EOF' > /root/mysql_secure.exp
    #!/usr/bin/expect -f
    set timeout -1
    set password [lindex $argv 0]
    
    spawn mysql_secure_installation
    
    expect "Enter current password for root (enter for none):"
    send "\r"
    expect "Switch to unix_socket authentication"
    send "Y\r"
    expect "Change the root password?"
    send "y\r"
    expect "New password:"
    send "$password\r"
    expect "Re-enter new password:"
    send "$password\r"
    expect "Remove anonymous users?"
    send "Y\r"
    expect "Disallow root login remotely?"
    send "Y\r"
    expect "Remove test database and access to it?"
    send "Y\r"
    expect "Reload privilege tables now?"
    send "Y\r"
    expect eof
    EOF
    
    chmod +x /root/mysql_secure.exp
    /usr/bin/expect /root/mysql_secure.exp "$MYSQL_ROOT_PASSWORD"
    echo "✅ MariaDB 초기화 및 비밀번호 설정 완료!"
    
    # 6️⃣ phpMyAdmin 설치 및 설정
    echo ">>> phpMyAdmin 설치 중..."
    dnf install -y phpmyadmin
    
    cat <<EOF > /etc/httpd/conf.d/phpMyAdmin.conf
    Alias /phpMyAdmin /usr/share/phpMyAdmin
    Alias /phpmyadmin /usr/share/phpMyAdmin
    
    <Directory /usr/share/phpMyAdmin/>
       AddDefaultCharset UTF-8
       Require all granted
    </Directory>
    EOF
    
    echo "✅ phpMyAdmin 설정 완료!"
    
    # 7️⃣ Apache 재시작
    systemctl restart httpd
    
    # 8️⃣ 요약 출력
    echo "🎉 LAMP 환경 설치 완료!"
    echo "-------------------------------------"
    echo "🌐 Apache : 실행 중 (http://서버IP)"
    echo "🐘 PHP : 8.4 (Remi 저장소)"
    echo "🗄️  MariaDB : root 비밀번호 → $MYSQL_ROOT_PASSWORD"
    echo "🧩 phpMyAdmin : http://서버IP/phpmyadmin"
    echo "⚙️ SELinux 비활성화 (재부팅 후 적용)"
    echo "🚫 firewalld 중지 완료"
    echo "-------------------------------------"
    """


bash_script = php_mariaDB()

run_remote(bash_script)

Last login: Mon Oct 27 17:56:01 2025 from 172.16.16.1
set -e

    echo "===== LAMP (Apache + PHP + MariaDB + phpMyAdmin) 자동 설치 시작 ====="

    # 0️⃣ SELinux & firewalld 비활성화
    echo ">>> SELinux & firewalld 비활성화 중..."
    sed -i 's/^SELINUX=.*/SELINUX=disabled/' /etc/selinux/config
    systemctl stop firewalld
    systemctl disable firewalld
    echo "✅ SELinux 비활성화 (재부팅 후 적용), firewalld 중지 완료!"

    # 1️⃣ 필수 패키지 설치
    echo ">>> DNF 플러그인 설치..."
    dnf -y install dnf-plugins-core epel-release expect

    # 2️⃣ PHP 리포지토리 등록 및 활성화
    echo ">>> Remi 리포지토리 설정 중..."
    dnf install -y https://rpms.remirepo.net/enterprise/remi-release-9.rpm
    dnf module reset php -y
    dnf module enable php:remi-8.4 -y
    echo "✅ PHP 8.4 모듈 활성화 완료!"

    # 3️⃣ Apache + PHP 설치
    echo ">>> Apache + PHP 설치 중..."
    dnf -y install httpd php php-fpm php-mysqlnd php-cli php-common php-devel         php-mbstring php-xml php-gd php-json php-intl php-zip php-opcache

    systemctl enable --now httpd php-fpm
